In [2]:
# Module importing
import pandas as pd

#Dataset Path
DATASET_PATH = "./data/"

In [3]:
# Importing dataset

# NOTA: hay que hacer un pip install pyarrow para que funcione

df_feb= pd.read_parquet(DATASET_PATH+'yellow_tripdata_2020-02.parquet', engine='pyarrow')
df_aug= pd.read_parquet(DATASET_PATH+'yellow_tripdata_2020-08.parquet', engine='pyarrow')
df_nov= pd.read_parquet(DATASET_PATH+'yellow_tripdata_2020-11.parquet', engine='pyarrow')

# 1- Analisis exploratorio inicial

## 1.1 - Dataset features and characteristics

In [7]:
df_feb.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2020-02-01 00:17:35,2020-02-01 00:30:32,1.0,2.60,1.0,N,145,7,1,11.0,0.5,0.5,2.45,0.0,0.3,14.75,0.0,None
1,1,2020-02-01 00:32:47,2020-02-01 01:05:36,1.0,4.80,1.0,N,45,61,1,21.5,3.0,0.5,6.30,0.0,0.3,31.60,2.5,None
2,1,2020-02-01 00:31:44,2020-02-01 00:43:28,1.0,3.20,1.0,N,186,140,1,11.0,3.0,0.5,1.00,0.0,0.3,15.80,2.5,None
3,2,2020-02-01 00:07:35,2020-02-01 00:31:39,1.0,4.38,1.0,N,144,140,1,18.0,0.5,0.5,3.00,0.0,0.3,24.80,2.5,None
4,2,2020-02-01 00:51:43,2020-02-01 01:01:29,1.0,2.28,1.0,N,238,152,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80,0.0,None


In [8]:
df_feb.tail()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
6299362,2,2020-02-29 23:24:00,2020-02-29 23:55:00,NaN,10.59,NaN,None,225,205,0,48.85,2.75,0.5,0.0,0.00,0.3,52.40,NaN,None
6299363,6,2020-02-29 23:02:01,2020-02-29 23:02:25,NaN,5.88,NaN,None,265,226,0,26.65,2.33,0.5,0.0,0.00,0.3,32.28,NaN,None
6299364,2,2020-02-29 23:35:00,2020-02-29 23:58:00,NaN,4.46,NaN,None,45,80,0,24.70,2.75,0.5,0.0,0.00,0.3,28.25,NaN,None
6299365,2,2020-02-29 23:42:00,2020-03-01 00:11:00,NaN,15.73,NaN,None,235,197,0,53.03,2.75,0.5,0.0,6.12,0.3,62.70,NaN,None
6299366,6,2020-02-29 23:02:13,2020-03-01 00:03:24,NaN,11.34,NaN,None,265,116,0,37.62,3.29,0.5,0.0,0.00,0.3,44.21,NaN,None


## 1.2 Tipos de datos y tipos de variables

In [4]:
df_feb.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                      object
dtype: object

In [5]:
df_feb.nunique()

VendorID                       4
tpep_pickup_datetime     2027136
tpep_dropoff_datetime    2031299
passenger_count               10
trip_distance               4391
RatecodeID                     7
store_and_fwd_flag             2
PULocationID                 259
DOLocationID                 261
payment_type                   5
fare_amount                 5137
extra                        122
mta_tax                       11
tip_amount                  3680
tolls_amount                 937
improvement_surcharge          3
total_amount               12311
congestion_surcharge           7
airport_fee                    0
dtype: int64

## 1.2.1 Tipos de variables del dataset

Se analiza la descripcion de las features del dataset provistas por el proveedor del dataset. 
File: data_dictionary_trip_records_yellow.pdf

### Numericas Discretas (variable no puede tomar valores entre dos valores contiguos)
passenger_count --------------------- float64  //los pasajeros saltan de a 1 <br>
tolls_amount --------------------- float64 <br>
% es funcion de variables discretas es decir cobinacion lineal de 0,1,2,3 etc peajes que hubo durante el viaje.  <br> <br> 


### Numericas Continuas (variable puede tomar valores en un rango)
trip_distance --------------------- float64<br>
fare_amount --------------------- float64<br> 
tip_amount --------------------- loat64<br>
total_amount --------------------- float64<br>

### Categoricas Nominales (variable qualitativa, pone un nombre/label sin orden interinseco entre valores)
VendorID --------------------- int64 --------------------- Cardinalidad:2 // nombre de empresa<br>
congestion_surcharge --------------------- float64 --------------------- Cardinalidad:2 // se cobra o no<br>
extra --------------------- float64 --------------------- Cardinalidad:2 // tres categorias disitntas<br> 

payment_type --------------------- int64 --------------------- Cardinalidad:6 // forma de pago<br> 
airport_fee  --------------------- object --------------------- Cardinalidad:2 // se cobra o no<br>
mta_tax --------------------- float64 --------------------- Cardinalidad:2 // se cobra o no <br>
improvement_surcharge  --------------------- float64 --------------------- Cardinalidad:2 //se cobra o no <br>
store_and_fwd_flag --------------------- object --------------------- Cardinalidad:2 // dos categorias <br>
RatecodeID --------------------- float64 --------------------- Cardinalidad:6 // ID de la tarifa seleccionada al momento de terminar el viaje <br>


### categoricas ordinales (variable qualitativa, pone un nombre/label CON orden interinseco entre valores)
PULocationID --------------------- int64 --------------------- Cardinalidad:2 // nombre, orden es localizaion geografica<br> 
DOLocationID --------------------- int64 --------------------- Cardinalidad:2 // nombre, orden es localizaion geografica<br>


### Fecha y hora
tpep_pickup_datetime --------------------- datetime64[ns]<br>
tpep_dropoff_datetime --------------------- datetime64[ns]<br>

### Compuestas
//no hay

## que se va a responder del dataset (parte del entregable)

## 1.2.2 Identificacion de variables de entrada y de salida

Variable de salida : total_amount, congestion_surcharge,fare_amount,tolls_amount
Variables independientes:
Variables categoricas: No hay categorias en las columnas(ej. meses, nombres propios, etc)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2020-02-28 17:55:01,2020-02-23,1.0,0.8,1.0,N,161,236,1,6.0,0.0,0.5,0.0,0.0,0.3,10.3,2.5,NaN


## 1.3 Handling missing data

In [ ]:
df_feb.isna().values.any()

In [ ]:
pd.isna(df_feb)

Analisis: tenemos missing values. de los siguientes tipos:
- NaNs  ver tail congestion_surcharge
- None  ver tail airport_fee

In [ ]:
df_feb.isnull().sum()

# Annex 1- tipos de variables
Tipos de variables 

Categóricas. Datos con un número limitado de valores o categorías distintas (por ejemplo, género o religión). Las variables categóricas pueden ser variables de cadena (alfanuméricas) o variables numéricas que utilizan códigos numéricos para representar a categorías (por ejemplo, 0 = hombre y 1 = mujer). También se hace referencia a estos datos como datos cualitativos. Las variables categóricas pueden ser nominales u ordinales

Nominal. Una variable puede tratarse como nominal cuando sus valores representan categorías sin clasificación intrínseca (por ejemplo, el departamento de la empresa en la que trabaja un empleado). Ejemplos de variables nominales incluyen la región, el código postal y la afiliación religiosa.

Ordinal. Una variable puede ser tratada como ordinal cuando sus valores representan categorías con alguna clasificación intrínseca (por ejemplo, niveles de satisfacción de servicio de altamente insatisfecho a altamente satisfecho). Ejemplos de variables ordinales incluyen puntuaciones de actitud que representan grado de satisfacción o confianza y puntuaciones de calificación de preferencia.


CATEGORICAL VARIABLES: In statistics, a categorical variable (also called qualitative variable) is a variable that can take on one of a limited, and usually fixed, number of possible values, assigning each individual or other unit of observation to a particular group or nominal category on the basis of some qualitative property

NOMINAL VARIABLES: A nominal category or a nominal group is a group of objects or ideas that can be collectively grouped on the basis of a particular characteristic—a qualitative property. A variable that codes whether each one in a set of observations is in a particular nominal category is called a categorical variable. 

ORDINAL VARIABLES: Ordinal data is a categorical, statistical data type where the variables have natural, ordered categories and the distances between the categories are not known.